# Import all important libraries

In [45]:
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas_profiling
import datetime
import pytz
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import utils
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import ShuffleSplit, train_test_split 
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from datetime import date
from io import StringIO # python3; python2: BytesIO 
import boto3
#import vpython as vs

## Read the csv

In [46]:
dedham = pd.read_csv('dedhamdata.csv')

In [47]:
dedham.head(10)

,dateProduced,id,itemName,itemType,clientId,loggedTime,meal,actionTaken,actionReason,quantity,panId,panWeight,unit,station,employeeName,locationId
0,2019-09-16,39105,"Bacon, Maple Dry Rub Wellshire CE",Menu Item,NaN,2019-09-16T13:15:15.000Z,Breakfast,Discarded,Over Produced,0.110231,NaN,NaN,Lbs,NaN,NaN,51
1,2019-09-16,39106,CheeseLasagna,Menu Item,NaN,2019-09-16T13:15:28.000Z,Breakfast,Discarded,Over Produced,0.110231,NaN,NaN,Lbs,NaN,NaN,51
2,2019-09-16,39107,"Bacon, Maple Dry Rub Wellshire CE",Menu Item,NaN,2019-09-16T13:15:47.000Z,Breakfast,Discarded,Over Produced,5.467460,NaN,NaN,Lbs,NaN,NaN,51
3,2019-09-16,39108,ChickenTikkaMasala,Menu Item,NaN,2019-09-16T13:15:59.000Z,Breakfast,Discarded,Over Produced,5.467460,NaN,NaN,Lbs,NaN,NaN,51
4,2019-09-16,39109,ChickenTikkaMasala,Menu Item,NaN,2019-09-16T13:23:45.000Z,Breakfast,Discarded,Over Produced,0.000000,NaN,NaN,Lbs,NaN,NaN,51
5,2019-09-16,39110,"Bacon, Maple Dry Rub Wellshire CE",Menu Item,NaN,2019-09-16T13:23:58.000Z,Breakfast,Discarded,Over Produced,0.000000,NaN,NaN,Lbs,NaN,NaN,51
6,2019-09-16,39111,"Bacon, Maple Dry Rub Wellshire CE",Menu Item,NaN,2019-09-16T13:25:26.000Z,Breakfast,Discarded,Over Produced,5.467460,NaN,NaN,Lbs,NaN,NaN,51
7,2019-09-16,39112,KaleWhiteBeanSalad,Menu Item,NaN,2019-09-16T13:29:14.000Z,Breakfast,Discarded,Over Produced,1.962110,NaN,NaN,Lbs,NaN,NaN,51
8,2019-09-16,39113,VegtableKorma,Menu Item,NaN,2019-09-16T13:29:48.000Z,Breakfast,Discarded,Over Produced,0.000000,NaN,NaN,Lbs,NaN,NaN,51
9,2019-09-16,39114,VegtableKorma,Menu Item,NaN,2019-09-16T13:30:12.000Z,Breakfast,Discarded,Over Produced,0.000000,NaN,NaN,Lbs,NaN,NaN,51


In [48]:
#pandas_profiling.ProfileReport(dedham)

# Insights from Dedham Raw Data

dateproduced needs to be changed into datetime format.  
Id is distinct for all rows.  
itemName has 144 distinct counts which contributes to 9.6% of total data. Chicken tender is mostly logged itemName.  
itemType is constant which is Menu Item.  
ClientId is missing for 189 rows which contribute 12.5% missing values of the total and has 139 distinct counts which contribute to 9.2% of total.  
loggedTime has 1504 distinct count which is 99.8% of total datta, we can say it is unique. Need to convert it to datetime function.  
meal has constant value breakfast  
actionTaken is categorical with 4 distinct values named Served, Discarded, Saved, Donate with counts 747,464,293,3 repectively.  
actionReason is categorical with 5 distinct values named Start, Overproduced, End, Refill, Spoil with counts 503,464,293,244,3 respectively.  
quantity has 676 distinct values with 44.9% uniqueness.  
panId has 10 distinct count with 43.1%(650 counts) missing data.  
panWeight has 198(13.1% of all) zeros and 189(12.5%) missing data..  
unit has constant value Lbs.  
station has constant value nan.  
employeeName has constant value nan.  
locationId is constant 51.  
Thus, we remove itemType, meal, unit, station, employeeName, locationId


In [49]:
dedham.head()

,dateProduced,id,itemName,itemType,clientId,loggedTime,meal,actionTaken,actionReason,quantity,panId,panWeight,unit,station,employeeName,locationId
0,2019-09-16,39105,"Bacon, Maple Dry Rub Wellshire CE",Menu Item,NaN,2019-09-16T13:15:15.000Z,Breakfast,Discarded,Over Produced,0.110231,NaN,NaN,Lbs,NaN,NaN,51
1,2019-09-16,39106,CheeseLasagna,Menu Item,NaN,2019-09-16T13:15:28.000Z,Breakfast,Discarded,Over Produced,0.110231,NaN,NaN,Lbs,NaN,NaN,51
2,2019-09-16,39107,"Bacon, Maple Dry Rub Wellshire CE",Menu Item,NaN,2019-09-16T13:15:47.000Z,Breakfast,Discarded,Over Produced,5.467460,NaN,NaN,Lbs,NaN,NaN,51
3,2019-09-16,39108,ChickenTikkaMasala,Menu Item,NaN,2019-09-16T13:15:59.000Z,Breakfast,Discarded,Over Produced,5.467460,NaN,NaN,Lbs,NaN,NaN,51
4,2019-09-16,39109,ChickenTikkaMasala,Menu Item,NaN,2019-09-16T13:23:45.000Z,Breakfast,Discarded,Over Produced,0.000000,NaN,NaN,Lbs,NaN,NaN,51


## Change dateProduced and loggedTime to datetime format

In [50]:
dedham['dateProduced'] = pd.to_datetime(dedham['dateProduced'])
dedham['loggedTime'] = pd.to_datetime(dedham['loggedTime'])

In [51]:
dedham.head()

,dateProduced,id,itemName,itemType,clientId,loggedTime,meal,actionTaken,actionReason,quantity,panId,panWeight,unit,station,employeeName,locationId
0,2019-09-16,39105,"Bacon, Maple Dry Rub Wellshire CE",Menu Item,NaN,2019-09-16 13:15:15+00:00,Breakfast,Discarded,Over Produced,0.110231,NaN,NaN,Lbs,NaN,NaN,51
1,2019-09-16,39106,CheeseLasagna,Menu Item,NaN,2019-09-16 13:15:28+00:00,Breakfast,Discarded,Over Produced,0.110231,NaN,NaN,Lbs,NaN,NaN,51
2,2019-09-16,39107,"Bacon, Maple Dry Rub Wellshire CE",Menu Item,NaN,2019-09-16 13:15:47+00:00,Breakfast,Discarded,Over Produced,5.467460,NaN,NaN,Lbs,NaN,NaN,51
3,2019-09-16,39108,ChickenTikkaMasala,Menu Item,NaN,2019-09-16 13:15:59+00:00,Breakfast,Discarded,Over Produced,5.467460,NaN,NaN,Lbs,NaN,NaN,51
4,2019-09-16,39109,ChickenTikkaMasala,Menu Item,NaN,2019-09-16 13:23:45+00:00,Breakfast,Discarded,Over Produced,0.000000,NaN,NaN,Lbs,NaN,NaN,51


## Drop unnecessary items

In [52]:
dedham = dedham.drop(['itemType','meal','unit','station','employeeName','locationId','clientId','panId'], axis=1)

In [53]:
dedham.head()

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight
0,2019-09-16,39105,"Bacon, Maple Dry Rub Wellshire CE",2019-09-16 13:15:15+00:00,Discarded,Over Produced,0.110231,NaN
1,2019-09-16,39106,CheeseLasagna,2019-09-16 13:15:28+00:00,Discarded,Over Produced,0.110231,NaN
2,2019-09-16,39107,"Bacon, Maple Dry Rub Wellshire CE",2019-09-16 13:15:47+00:00,Discarded,Over Produced,5.467460,NaN
3,2019-09-16,39108,ChickenTikkaMasala,2019-09-16 13:15:59+00:00,Discarded,Over Produced,5.467460,NaN
4,2019-09-16,39109,ChickenTikkaMasala,2019-09-16 13:23:45+00:00,Discarded,Over Produced,0.000000,NaN


In [54]:
dedham['loggedDate'] = dedham['loggedTime'].dt.date
dedham['loggedTime'] = dedham['loggedTime'].dt.time

In [55]:
dedham

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate
0,2019-09-16,39105,"Bacon, Maple Dry Rub Wellshire CE",13:15:15,Discarded,Over Produced,0.110231,NaN,2019-09-16
1,2019-09-16,39106,CheeseLasagna,13:15:28,Discarded,Over Produced,0.110231,NaN,2019-09-16
2,2019-09-16,39107,"Bacon, Maple Dry Rub Wellshire CE",13:15:47,Discarded,Over Produced,5.467460,NaN,2019-09-16
3,2019-09-16,39108,ChickenTikkaMasala,13:15:59,Discarded,Over Produced,5.467460,NaN,2019-09-16
4,2019-09-16,39109,ChickenTikkaMasala,13:23:45,Discarded,Over Produced,0.000000,NaN,2019-09-16
...,...,...,...,...,...,...,...,...,...
2130,2019-10-01,44329,Turmeric Kale Rice,21:53:22,Discarded,Over Produced,3.218750,1.59,2019-10-01
2131,2019-10-01,44330,Curried Cauliflower,21:53:32,Discarded,Over Produced,1.895980,1.59,2019-10-01
2132,2019-10-01,44332,Buffalo Wings,21:53:52,Discarded,Over Produced,7.142980,1.59,2019-10-01
2133,2019-10-01,44333,Bangkok Wings,21:54:05,Discarded,Over Produced,8.399610,1.59,2019-10-01


### So We actually started keeping track on the spoilage since 24th September. Thus, providing data from 24th could be better.

In [56]:
dedham = dedham[(dedham['loggedDate'] >= datetime.date(year=2019,month=9,day=30))]

In [57]:
dedham

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate
1671,2019-09-29,42074,Grape Tomatoes,07:08:28,Served,Start,15.03550,0.35,2019-09-30
1672,2019-09-29,42078,Roasted Garlic,07:11:14,Served,Start,14.50640,0.35,2019-09-30
1673,2019-09-29,42079,Roasted Mushrooms,07:11:31,Served,Start,14.55050,0.35,2019-09-30
1674,2019-09-29,42080,Hard Boiled Eggs,07:11:53,Served,Start,15.87330,0.35,2019-09-30
1675,2019-09-29,42081,Red Quinoa,07:12:17,Served,Start,13.80090,0.35,2019-09-30
...,...,...,...,...,...,...,...,...,...
2130,2019-10-01,44329,Turmeric Kale Rice,21:53:22,Discarded,Over Produced,3.21875,1.59,2019-10-01
2131,2019-10-01,44330,Curried Cauliflower,21:53:32,Discarded,Over Produced,1.89598,1.59,2019-10-01
2132,2019-10-01,44332,Buffalo Wings,21:53:52,Discarded,Over Produced,7.14298,1.59,2019-10-01
2133,2019-10-01,44333,Bangkok Wings,21:54:05,Discarded,Over Produced,8.39961,1.59,2019-10-01


In [58]:
#pandas_profiling.ProfileReport(dedham)

In [59]:
#dedham.to_excel("data.xlsx", sheet_name='dedham')

In [60]:
dedham

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate
1671,2019-09-29,42074,Grape Tomatoes,07:08:28,Served,Start,15.03550,0.35,2019-09-30
1672,2019-09-29,42078,Roasted Garlic,07:11:14,Served,Start,14.50640,0.35,2019-09-30
1673,2019-09-29,42079,Roasted Mushrooms,07:11:31,Served,Start,14.55050,0.35,2019-09-30
1674,2019-09-29,42080,Hard Boiled Eggs,07:11:53,Served,Start,15.87330,0.35,2019-09-30
1675,2019-09-29,42081,Red Quinoa,07:12:17,Served,Start,13.80090,0.35,2019-09-30
...,...,...,...,...,...,...,...,...,...
2130,2019-10-01,44329,Turmeric Kale Rice,21:53:22,Discarded,Over Produced,3.21875,1.59,2019-10-01
2131,2019-10-01,44330,Curried Cauliflower,21:53:32,Discarded,Over Produced,1.89598,1.59,2019-10-01
2132,2019-10-01,44332,Buffalo Wings,21:53:52,Discarded,Over Produced,7.14298,1.59,2019-10-01
2133,2019-10-01,44333,Bangkok Wings,21:54:05,Discarded,Over Produced,8.39961,1.59,2019-10-01


In [61]:
dedham['netquant'] = dedham['quantity'] - dedham['panWeight']

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
dedham

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate,netquant
1671,2019-09-29,42074,Grape Tomatoes,07:08:28,Served,Start,15.03550,0.35,2019-09-30,14.68550
1672,2019-09-29,42078,Roasted Garlic,07:11:14,Served,Start,14.50640,0.35,2019-09-30,14.15640
1673,2019-09-29,42079,Roasted Mushrooms,07:11:31,Served,Start,14.55050,0.35,2019-09-30,14.20050
1674,2019-09-29,42080,Hard Boiled Eggs,07:11:53,Served,Start,15.87330,0.35,2019-09-30,15.52330
1675,2019-09-29,42081,Red Quinoa,07:12:17,Served,Start,13.80090,0.35,2019-09-30,13.45090
...,...,...,...,...,...,...,...,...,...,...
2130,2019-10-01,44329,Turmeric Kale Rice,21:53:22,Discarded,Over Produced,3.21875,1.59,2019-10-01,1.62875
2131,2019-10-01,44330,Curried Cauliflower,21:53:32,Discarded,Over Produced,1.89598,1.59,2019-10-01,0.30598
2132,2019-10-01,44332,Buffalo Wings,21:53:52,Discarded,Over Produced,7.14298,1.59,2019-10-01,5.55298
2133,2019-10-01,44333,Bangkok Wings,21:54:05,Discarded,Over Produced,8.39961,1.59,2019-10-01,6.80961


In [63]:
#dedham.to_excel("data.xlsx", sheet_name='dedham')

In [64]:
#pandas_profiling.ProfileReport(dedham)

In [65]:
dedham[dedham['netquant']<0]

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate,netquant
1787,2019-09-30,42437,Turkey Bacon,12:03:15,Discarded,Over Produced,2.13848,2.35,2019-09-30,-0.21152
1803,2019-09-30,42480,Romaine Lettuce,12:37:22,Served,Start,2.07235,2.11,2019-09-30,-0.03765
1807,2019-09-30,42493,Arugula,12:44:37,Served,Start,1.03617,2.11,2019-09-30,-1.07383
1818,2019-09-30,42545,Lentil Dal & Spinach Soup,13:30:08,Discarded,Over Produced,1.85188,2.11,2019-09-30,-0.25812
1843,2019-09-30,42710,Olive Tomato Parsley Pollock,16:15:45,Discarded,Over Produced,2.27076,2.35,2019-09-30,-0.07924
1875,2019-09-30,42848,Sublime Swine Smoked Pulled Pork,19:33:38,Discarded,Over Produced,2.93215,3.46,2019-09-30,-0.52785
1876,2019-09-30,42883,Curried Cauliflower,20:37:24,Discarded,Over Produced,1.19050,1.59,2019-09-30,-0.39950
2024,2019-10-01,43691,Sauteed Kale w/ Mushrooms,12:27:30,Discarded,Over Produced,2.77782,3.46,2019-10-01,-0.68218
2070,2019-10-01,43960,Beef Chili,16:46:47,Discarded,Over Produced,2.24872,2.35,2019-10-01,-0.10128
2081,2019-10-01,44015,Lentil Dal & Spinach Soup,17:57:52,Discarded,Over Produced,2.09439,2.35,2019-10-01,-0.25561


In [66]:
dedham['status'] = np.where(dedham['netquant'] < 0 , 'Flag', 'Clean')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [67]:
dedham

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate,netquant,status
1671,2019-09-29,42074,Grape Tomatoes,07:08:28,Served,Start,15.03550,0.35,2019-09-30,14.68550,Clean
1672,2019-09-29,42078,Roasted Garlic,07:11:14,Served,Start,14.50640,0.35,2019-09-30,14.15640,Clean
1673,2019-09-29,42079,Roasted Mushrooms,07:11:31,Served,Start,14.55050,0.35,2019-09-30,14.20050,Clean
1674,2019-09-29,42080,Hard Boiled Eggs,07:11:53,Served,Start,15.87330,0.35,2019-09-30,15.52330,Clean
1675,2019-09-29,42081,Red Quinoa,07:12:17,Served,Start,13.80090,0.35,2019-09-30,13.45090,Clean
...,...,...,...,...,...,...,...,...,...,...,...
2130,2019-10-01,44329,Turmeric Kale Rice,21:53:22,Discarded,Over Produced,3.21875,1.59,2019-10-01,1.62875,Clean
2131,2019-10-01,44330,Curried Cauliflower,21:53:32,Discarded,Over Produced,1.89598,1.59,2019-10-01,0.30598,Clean
2132,2019-10-01,44332,Buffalo Wings,21:53:52,Discarded,Over Produced,7.14298,1.59,2019-10-01,5.55298,Clean
2133,2019-10-01,44333,Bangkok Wings,21:54:05,Discarded,Over Produced,8.39961,1.59,2019-10-01,6.80961,Clean


In [68]:
dedham[dedham['netquant']<0].count()

dateProduced    13
id              13
itemName        13
loggedTime      13
actionTaken     13
actionReason    13
quantity        13
panWeight       13
loggedDate      13
netquant        13
status          13
dtype: int64

In [69]:
dedhamuniqueitems = dedham.itemName.unique()

print(dedhamuniqueitems)
np.size(dedhamuniqueitems)


['Grape Tomatoes ' 'Roasted Garlic ' 'Roasted Mushrooms '
 'Hard Boiled Eggs ' 'Red Quinoa' 'Tortellini & Sopressata '
 'Garlic Parmesan Pasta' 'Roman Cappellini Pasta Salad '
 'Bistro Pasta Salad ' 'Cantalope' 'Blueberries' 'Red Grapes '
 'Seasonal Citrus ' 'Baby Spinach' 'Tofu Breakfast Scramble' 'Plantains'
 'Roasted Sweet Potatoes' 'Sausage Patties ' 'Turkey Bacon ' 'Bacon '
 'Blueberry Lemon Pancakes' 'Black Beans' 'Garbanzo Beans' 'Farro' 'Corn '
 'Peas' 'Kidney Beans' 'Golden Beets' 'Sweet & Spicy Kale Pad Thai'
 'Chicken Pasta Caesar Salad' 'Ham & Cheese Omelet'
 'Fennel Pistachio Lemon Vin Salad' 'Scrambled Eggs' 'Berry Power Salad'
 'Breakfast Potatoes' 'Spinach Feta Orzo Salad'
 'Egg White & Spinach Scramble' 'Spring Mix' 'Blackberries' 'Zucchini '
 'Mixed Peppers' 'Snap Peas' 'Chickpea Asparagus Radicchio '
 'Golden Sesame Tofu ' 'Kale & Brussel Sprout Salad'
 'Chinese Black Pepper Green Beans ' 'Lemon Artichoke Slaw'
 'Kale White Bean Salad ' 'New England Clam Chowder' 'Tu

113

In [70]:
dedhaminfo = pd.DataFrame(columns = ['itemname'])
dedhaminfo['itemname'] = dedhamuniqueitems

In [71]:
dedhaminfo

,itemname
0,Grape Tomatoes
1,Roasted Garlic
2,Roasted Mushrooms
3,Hard Boiled Eggs
4,Red Quinoa
...,...
108,"Chicken, Spicy Kerala Pepper"
109,Olive Tomato Parsley Pollock
110,Chicken & Scallion Salad
111,Edamame


In [72]:
dedhaminfo['location'] = 'Dedham'

In [73]:
uniqueitemsnetquant = []
for i in range(len(dedhamuniqueitems)):
    uniqueitemsnetquant.append((dedham[dedham.itemName==dedhamuniqueitems[i] ]).netquant.sum())

In [74]:
dedhaminfo['Total Quantity Weighted'] = uniqueitemsnetquant
dedhaminfo

,itemname,location,Total Quantity Weighted
0,Grape Tomatoes,Dedham,14.68550
1,Roasted Garlic,Dedham,24.76000
2,Roasted Mushrooms,Dedham,25.70800
3,Hard Boiled Eggs,Dedham,27.73630
4,Red Quinoa,Dedham,22.26880
...,...,...,...
108,"Chicken, Spicy Kerala Pepper",Dedham,37.69400
109,Olive Tomato Parsley Pollock,Dedham,16.42851
110,Chicken & Scallion Salad,Dedham,20.59060
111,Edamame,Dedham,4.94730


In [75]:
uniqueitemsputout = []
for i in range(len(dedhamuniqueitems)):
    uniqueitemsputout.append((dedham[(dedham.itemName==dedhamuniqueitems[i]) & (dedham.actionTaken=='Served')]).netquant.sum())

In [76]:
dedhaminfo['Total Quantity Put Out'] = uniqueitemsputout
dedhaminfo

,itemname,location,Total Quantity Weighted,Total Quantity Put Out
0,Grape Tomatoes,Dedham,14.68550,14.68550
1,Roasted Garlic,Dedham,24.76000,24.76000
2,Roasted Mushrooms,Dedham,25.70800,25.70800
3,Hard Boiled Eggs,Dedham,27.73630,27.73630
4,Red Quinoa,Dedham,22.26880,22.26880
...,...,...,...,...
108,"Chicken, Spicy Kerala Pepper",Dedham,37.69400,33.90739
109,Olive Tomato Parsley Pollock,Dedham,16.42851,16.07217
110,Chicken & Scallion Salad,Dedham,20.59060,20.59060
111,Edamame,Dedham,4.94730,4.94730


In [77]:
uniqueitemsshrink = []
for i in range(len(dedhamuniqueitems)):
    uniqueitemsshrink.append((dedham[(dedham.itemName==dedhamuniqueitems[i]) & (dedham.actionTaken=='Discarded')]).netquant.sum())

In [78]:
dedhaminfo['Total Quantity Shrink'] = uniqueitemsshrink
dedhaminfo

,itemname,location,Total Quantity Weighted,Total Quantity Put Out,Total Quantity Shrink
0,Grape Tomatoes,Dedham,14.68550,14.68550,0.00000
1,Roasted Garlic,Dedham,24.76000,24.76000,0.00000
2,Roasted Mushrooms,Dedham,25.70800,25.70800,0.00000
3,Hard Boiled Eggs,Dedham,27.73630,27.73630,0.00000
4,Red Quinoa,Dedham,22.26880,22.26880,0.00000
...,...,...,...,...,...
108,"Chicken, Spicy Kerala Pepper",Dedham,37.69400,33.90739,3.78661
109,Olive Tomato Parsley Pollock,Dedham,16.42851,16.07217,0.35634
110,Chicken & Scallion Salad,Dedham,20.59060,20.59060,0.00000
111,Edamame,Dedham,4.94730,4.94730,0.00000


In [79]:
dedhaminfo['Total Quantity Sold'] = dedhaminfo['Total Quantity Put Out'] - dedhaminfo['Total Quantity Shrink']
dedhaminfo

,itemname,location,Total Quantity Weighted,Total Quantity Put Out,Total Quantity Shrink,Total Quantity Sold
0,Grape Tomatoes,Dedham,14.68550,14.68550,0.00000,14.68550
1,Roasted Garlic,Dedham,24.76000,24.76000,0.00000,24.76000
2,Roasted Mushrooms,Dedham,25.70800,25.70800,0.00000,25.70800
3,Hard Boiled Eggs,Dedham,27.73630,27.73630,0.00000,27.73630
4,Red Quinoa,Dedham,22.26880,22.26880,0.00000,22.26880
...,...,...,...,...,...,...
108,"Chicken, Spicy Kerala Pepper",Dedham,37.69400,33.90739,3.78661,30.12078
109,Olive Tomato Parsley Pollock,Dedham,16.42851,16.07217,0.35634,15.71583
110,Chicken & Scallion Salad,Dedham,20.59060,20.59060,0.00000,20.59060
111,Edamame,Dedham,4.94730,4.94730,0.00000,4.94730


In [80]:
writer = pd.ExcelWriter('data.xlsx', engine='xlsxwriter')
dedhaminfo.to_excel(writer, sheet_name = 'dedhamreportdemo')
dedham.to_excel(writer, sheet_name = 'dedham')
writer.save()
writer.close()
#dedhaminfo.to_excel("data.xlsx", sheet_name='dedhamreportdemo')

In [81]:
dedham09302019 = dedham[(dedham['loggedDate'] == datetime.date(year=2019,month=9,day=30))]
dedham10012019 = dedham[(dedham['loggedDate'] == datetime.date(year=2019,month=10,day=1))]

In [82]:
dedham09302019

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate,netquant,status
1671,2019-09-29,42074,Grape Tomatoes,07:08:28,Served,Start,15.03550,0.35,2019-09-30,14.68550,Clean
1672,2019-09-29,42078,Roasted Garlic,07:11:14,Served,Start,14.50640,0.35,2019-09-30,14.15640,Clean
1673,2019-09-29,42079,Roasted Mushrooms,07:11:31,Served,Start,14.55050,0.35,2019-09-30,14.20050,Clean
1674,2019-09-29,42080,Hard Boiled Eggs,07:11:53,Served,Start,15.87330,0.35,2019-09-30,15.52330,Clean
1675,2019-09-29,42081,Red Quinoa,07:12:17,Served,Start,13.80090,0.35,2019-09-30,13.45090,Clean
...,...,...,...,...,...,...,...,...,...,...,...
1904,2019-09-30,42999,Bangkok Wings,21:49:53,Discarded,Over Produced,8.26733,1.94,2019-09-30,6.32733,Clean
1905,2019-09-30,43001,Lemon Rice,21:50:11,Discarded,Over Produced,4.76198,1.94,2019-09-30,2.82198,Clean
1906,2019-09-30,43003,Buttered Cabbage,21:50:28,Discarded,Over Produced,5.24700,1.94,2019-09-30,3.30700,Clean
1907,2019-09-30,43005,Turmeric Kale Rice,21:50:44,Discarded,Over Produced,3.92423,1.94,2019-09-30,1.98423,Clean


In [83]:
dedham10012019

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate,netquant,status
1909,2019-09-30,43120,Spring Mix,06:18:52,Served,Start,6.45954,1.94,2019-10-01,4.51954,Clean
1910,2019-09-30,43122,Romaine Lettuce,06:19:06,Served,Start,6.45954,1.94,2019-10-01,4.51954,Clean
1911,2019-09-30,43123,Romaine Lettuce,06:19:20,Served,Start,6.34931,1.94,2019-10-01,4.40931,Clean
1912,2019-09-30,43124,Baby Spinach,06:19:35,Served,Start,6.76819,1.94,2019-10-01,4.82819,Clean
1913,2019-09-30,43194,Plantains,07:06:17,Served,Start,6.74615,1.94,2019-10-01,4.80615,Clean
...,...,...,...,...,...,...,...,...,...,...,...
2130,2019-10-01,44329,Turmeric Kale Rice,21:53:22,Discarded,Over Produced,3.21875,1.59,2019-10-01,1.62875,Clean
2131,2019-10-01,44330,Curried Cauliflower,21:53:32,Discarded,Over Produced,1.89598,1.59,2019-10-01,0.30598,Clean
2132,2019-10-01,44332,Buffalo Wings,21:53:52,Discarded,Over Produced,7.14298,1.59,2019-10-01,5.55298,Clean
2133,2019-10-01,44333,Bangkok Wings,21:54:05,Discarded,Over Produced,8.39961,1.59,2019-10-01,6.80961,Clean


In [84]:
dedham09302019

,dateProduced,id,itemName,loggedTime,actionTaken,actionReason,quantity,panWeight,loggedDate,netquant,status
1671,2019-09-29,42074,Grape Tomatoes,07:08:28,Served,Start,15.03550,0.35,2019-09-30,14.68550,Clean
1672,2019-09-29,42078,Roasted Garlic,07:11:14,Served,Start,14.50640,0.35,2019-09-30,14.15640,Clean
1673,2019-09-29,42079,Roasted Mushrooms,07:11:31,Served,Start,14.55050,0.35,2019-09-30,14.20050,Clean
1674,2019-09-29,42080,Hard Boiled Eggs,07:11:53,Served,Start,15.87330,0.35,2019-09-30,15.52330,Clean
1675,2019-09-29,42081,Red Quinoa,07:12:17,Served,Start,13.80090,0.35,2019-09-30,13.45090,Clean
...,...,...,...,...,...,...,...,...,...,...,...
1904,2019-09-30,42999,Bangkok Wings,21:49:53,Discarded,Over Produced,8.26733,1.94,2019-09-30,6.32733,Clean
1905,2019-09-30,43001,Lemon Rice,21:50:11,Discarded,Over Produced,4.76198,1.94,2019-09-30,2.82198,Clean
1906,2019-09-30,43003,Buttered Cabbage,21:50:28,Discarded,Over Produced,5.24700,1.94,2019-09-30,3.30700,Clean
1907,2019-09-30,43005,Turmeric Kale Rice,21:50:44,Discarded,Over Produced,3.92423,1.94,2019-09-30,1.98423,Clean


In [85]:
#pandas_profiling.ProfileReport(dedham09302019)

In [86]:

#dedham09302019[dedham09302019['actionReason']=='Start'].netquant.sum(axis = 0, skipna = True)

In [87]:
dedham09302019[dedham09302019['actionReason']=='Start'].netquant.sum()

1565.6865900000003

In [88]:
dedham09302019[dedham09302019['actionReason']=='Over Produced'].netquant.sum()

111.58751000000001